In [2]:
import os

import cv2 as cv
import numpy as np
from ultralytics import YOLO

model_path = os.path.join(
    os.path.dirname(os.getcwd()), "YOLO_model", "weights", "best.pt"
)
model = YOLO(model_path)

In [3]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [4]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape
    print(labels)
    print(res.boxes.conf)
    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [8]:
img = cv.imread("../scr.png")[:, :, :3]
img = cv.resize(img, (640, 640))
res = find_pieces(img)
import sys

sys.path.append("../src")
from analysis import get_fen

print(get_fen(res, "w"))
print(res)

0: 640x640 1 black-king, 7 black-pawns, 1 black-queen, 2 black-rooks, 1 white-bishop, 1 white-king, 6 white-pawns, 1 white-queen, 1 white-rook, 219.8ms
Speed: 5.5ms preprocess, 219.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


['black-queen', 'white-queen', 'white-bishop', 'black-king', 'black-pawn', 'white-king', 'black-pawn', 'white-rook', 'black-pawn', 'black-pawn', 'white-pawn', 'black-pawn', 'white-pawn', 'black-pawn', 'black-rook', 'black-pawn', 'white-pawn', 'white-pawn', 'white-pawn', 'white-pawn', 'black-rook']
tensor([0.9837, 0.9502, 0.9453, 0.9401, 0.9361, 0.9248, 0.9156, 0.9116, 0.8879, 0.8831, 0.8749, 0.8676, 0.8499, 0.8437, 0.8269, 0.8233, 0.8114, 0.7995, 0.7691, 0.7620, 0.6629])
1K2R3/1PP4P/P2BP3/5P2/6Q1/p1p1p1p1/1pq2p1p/1krr4 w
[['' 'K' '' '' 'R' '' '' '']
 ['' 'P' 'P' '' '' '' '' 'P']
 ['P' '' '' 'B' 'P' '' '' '']
 ['' '' '' '' '' 'P' '' '']
 ['' '' '' '' '' '' 'Q' '']
 ['p' '' 'p' '' 'p' '' 'p' '']
 ['' 'p' 'q' '' '' 'p' '' 'p']
 ['' 'k' 'r' 'r' '' '' '' '']]
